# Introduction to Pandas

**Pandas** is the standard package for working with data in Python. Here you learn the basics.

**Table of contents**<a id='toc0_'></a>    
- 1. [Pandas dataframe](#toc1_)    
- 2. [Indexing ("subsetting")](#toc2_)    
  - 2.1. [Logical and name-based indexing](#toc2_1_)    
  - 2.2. [Positional-indexing](#toc2_2_)    
- 3. [Adding a new variable](#toc3_)    
- 4. [Changing a subset of variables](#toc4_)    
- 5. [Summary](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
from IPython.display import display

## 1. <a id='toc1_'></a>[Pandas dataframe](#toc0_)

The `DataFrame` is the fundamental object in `pandas`. It is a matrix of data.

1. Each **column** is a variable (series) with a *name*.
1. Each **row** is an observation with an *index* (starting from 0).

In [2]:
X = pd.DataFrame({'id': [1,2,3], 
                  'inc': [11.7,13.9,14.6], 
                  'name': ['Vitus','Maximilian','Bo-bob']})

print(f'{type(X) = }')
print(f'{type(X['inc']) = }')
print(f'{type(X['inc'].dtype) = }')
display(X)

type(X) = <class 'pandas.core.frame.DataFrame'>
type(X['inc']) = <class 'pandas.core.series.Series'>
type(X['inc'].dtype) = <class 'numpy.dtypes.Float64DType'>


,id,inc,name
0,1,11.7,Vitus
1,2,13.9,Maximilian
2,3,14.6,Bo-bob


`.column` notation is also possible.

In [3]:
X.inc

0    11.7
1    13.9
2    14.6
Name: inc, dtype: float64

We can get (a view of) the underlying `numpy` data:

In [4]:
inc = X['inc'].to_numpy()
print(f'{type(inc) = }')
print(inc)

type(inc) = <class 'numpy.ndarray'>
[11.7 13.9 14.6]


**General information** can be obtained by

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3 non-null      int64  
 1   inc     3 non-null      float64
 2   name    3 non-null      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 204.0+ bytes


**What is `object`?** 

* In practice it is a `str` 
* Can contain multiple types (strings, floats, NaNs, etc.), which  can create difficulties

## 2. <a id='toc2_'></a>[Indexing ("subsetting")](#toc0_)

**Indexing**: Choosing a subset of the rows and/or columns of a dataframe.

### 2.1. <a id='toc2_1_'></a>[Logical and name-based indexing](#toc0_)

`.loc[]` for **logical** and **name-based** indexing.

**Syntax**: `df.loc[CONDITION,VARLIST]`, where 

- `CONDITION` is a vector of logical statements with the same length as the number of rows in the dataframe,
- `VARLIST` is a list over variables.

In [6]:
I = X['inc'] > 12 # boolean mask
cols = ['name'] # list of column names
X.loc[I,cols]

,name
1,Maximilian
2,Bo-bob


In [7]:
I = (X['inc'] > 12) & (X['inc'] < 14) # boolean mask
cols = ['id','name'] # list of column names
X.loc[I,cols]

,id,name
1,2,Maximilian


All columns is the default:

In [8]:
X.loc[I]

,id,inc,name
1,2,13.9,Maximilian


### 2.2. <a id='toc2_2_'></a>[Positional-indexing](#toc0_)

We can also use positional-indexing as in numpy arrays:

In [9]:
X.iloc[:2,1:] # two first row, all columns except the first

,inc,name
0,11.7,Vitus
1,13.9,Maximilian


## 3. <a id='toc3_'></a>[Adding a new variable](#toc0_)

Variables are added with `df['newvar'] = SOMETHING`. *The length must match or RHS is a scalar (broadcasting)*.

In [10]:
X['year'] = [2003, 2005, 2010]
X['zone'] = 7
X

,id,inc,name,year,zone
0,1,11.7,Vitus,2003,7
1,2,13.9,Maximilian,2005,7
2,3,14.6,Bo-bob,2010,7


The *something* can be an **expression based on other variables**.

In [11]:
X['inc_adj'] = X['inc'] - X['inc'].mean()
X

,id,inc,name,year,zone,inc_adj
0,1,11.7,Vitus,2003,7,-1.7
1,2,13.9,Maximilian,2005,7,0.5
2,3,14.6,Bo-bob,2010,7,1.2


In [12]:
def demean(inc):
    return inc-inc.mean()

X['inc_adj'] = demean(X.inc.to_numpy())
X

,id,inc,name,year,zone,inc_adj
0,1,11.7,Vitus,2003,7,-1.7
1,2,13.9,Maximilian,2005,7,0.5
2,3,14.6,Bo-bob,2010,7,1.2


**What if we have another dataframe with a new variable for a subset of the observations?**

In [13]:
Z = pd.DataFrame({'name':['Maximilian','Bo-bob'],'age': [20,30]})
Z

,name,age
0,Maximilian,20
1,Bo-bob,30


We can **join on name as index**:

In [14]:
Y = X.set_index('name')
Y['age'] = Z.set_index('name')
Y

,id,inc,year,zone,inc_adj,age
name,,,,,,
Vitus,1,11.7,2003,7,-1.7,NaN
Maximilian,2,13.9,2005,7,0.5,20.0
Bo-bob,3,14.6,2010,7,1.2,30.0


## 4. <a id='toc4_'></a>[Changing a subset of variables](#toc0_)

**Multiple rows, one value:**

In [15]:
Y = X.copy() # create copy of X to avoid overwriting it.
Y.loc[Y.id > 1, ['name']] = 'no name'
print('Y after change in names:')
Y

Y after change in names:


,id,inc,name,year,zone,inc_adj
0,1,11.7,Vitus,2003,7,-1.7
1,2,13.9,no name,2005,7,0.5
2,3,14.6,no name,2010,7,1.2


**LHS:** Selected using logical statement.<br>
**RHS:** Must either be:

1. a **single value** (all rows are set to this) 
2. a **list of values** with same length as the number of selected rows

**Multiple rows, multiple values:**

In [16]:
print('Original dataframe:')
Y = X.copy()
display(Y)

# subset the rows, where name is Vitus or year is 2010
I = (Y.name == 'Vitus') | (Y.year == 2010)

# print LHS
print('Subset (LHS in assignment):')
display(Y.loc[I,:])

# assignment
Y.loc[I, ['name']] = ['Bib', 'Peter']

print('Final dataframe:')
Y

Original dataframe:


,id,inc,name,year,zone,inc_adj
0,1,11.7,Vitus,2003,7,-1.7
1,2,13.9,Maximilian,2005,7,0.5
2,3,14.6,Bo-bob,2010,7,1.2


Subset (LHS in assignment):


,id,inc,name,year,zone,inc_adj
0,1,11.7,Vitus,2003,7,-1.7
2,3,14.6,Bo-bob,2010,7,1.2


Final dataframe:


,id,inc,name,year,zone,inc_adj
0,1,11.7,Bib,2003,7,-1.7
1,2,13.9,Maximilian,2005,7,0.5
2,3,14.6,Peter,2010,7,1.2


**Forbidden:** You cannot do the indexing on one code line and the change of data on another code line.

In [17]:
Y = X.copy()
Z = Y.loc[I] # return a copy
Z['name'] = ['Bib', 'Peter']

C:\Users\gmf123\AppData\Local\Temp\ipykernel_23752\2963678362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Z['name'] = ['Bib', 'Peter']


## 5. <a id='toc5_'></a>[Summary](#toc0_)

The main takeways are:

- Working with `pd.DataFrame` and its `.index`
- Indexing data (name-based with `.loc` or position-based with `.iloc`)
- Adding variables
- Changing subset of variables with `.loc`

**FYI:** Upcomming alternative to **pandas** is **polars**. See more [here](https://pola.rs/).

1. Faster
2. Simplier syntax (in my view)
3. Not fully integrate into other Python packages yet